In [2]:
import os
import io
from pydantic import BaseModel, ValidationError
from typing import List, Literal
from openai import OpenAI
from promptic import llm
from tenacity import retry, retry_if_exception_type
from dotenv import load_dotenv
import concurrent.futures as cf
from tempfile import NamedTemporaryFile

In [3]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [11]:
text = """
As Lorquas Ptomel raised his eyes to address the prisoner they fell on
me and he turned to Tars Tarkas with a word, and gesture of impatience.
Tars Tarkas made some reply which I could not catch, but which caused
Lorquas Ptomel to smile; after which they paid no further attention to
me.

“What is your name?” asked Lorquas Ptomel, addressing the prisoner.

“Dejah Thoris, daughter of Mors Kajak of Helium.”

“And the nature of your expedition?” he continued.

“It was a purely scientific research party sent out by my father’s
father, the Jeddak of Helium, to rechart the air currents, and to take
atmospheric density tests,” replied the fair prisoner, in a low,
well-modulated voice.

“We were unprepared for battle,” she continued, “as we were on a
peaceful mission, as our banners and the colors of our craft denoted.
The work we were doing was as much in your interests as in ours, for
you know full well that were it not for our labors and the fruits of
our scientific operations there would not be enough air or water on
Mars to support a single human life. For ages we have maintained the
air and water supply at practically the same point without an
appreciable loss, and we have done this in the face of the brutal and
ignorant interference of you green men.

“Why, oh, why will you not learn to live in amity with your fellows.
Must you ever go on down the ages to your final extinction but little
above the plane of the dumb brutes that serve you! A people without
written language, without art, without homes, without love; the victims
of eons of the horrible community idea. Owning everything in common,
even to your women and children, has resulted in your owning nothing in
common. You hate each other as you hate all else except yourselves.
Come back to the ways of our common ancestors, come back to the light
of kindliness and fellowship. The way is open to you, you will find the
hands of the red men stretched out to aid you. Together we may do still
more to regenerate our dying planet. The granddaughter of the greatest
and mightiest of the red jeddaks has asked you. Will you come?”

Lorquas Ptomel and the warriors sat looking silently and intently at
the young woman for several moments after she had ceased speaking. What
was passing in their minds no man may know, but that they were moved I
truly believe, and if one man high among them had been strong enough to
rise above custom, that moment would have marked a new and mighty era
for Mars.
"""

In [5]:
class DialogueItem(BaseModel):
    text: str
    speaker: Literal["female-1", "male-1", "female-2"]

    @property
    def voice(self):
        return {
            "female-1": "alloy",
            "male-1": "onyx",
            "female-2": "shimmer",
        }[self.speaker]


class Dialogue(BaseModel):
    scratchpad: str
    dialogue: List[DialogueItem]

In [12]:
@retry(retry=retry_if_exception_type(ValidationError))
@llm(model="gpt-4o-mini",)
def generate_dialogue(text: str) -> Dialogue:
    """
    Your task is to take the input text provided and turn it into an engaging, theater dialogue. You will have to identify what characters that take part of the text and what parts that are read by the Narrator. You should use this information to create a dialogue between the characters that is engaging, entertaining. The dialogue should be written in a natural, conversational tone and should flow smoothly from one character to the next. Be sure to include the characters' names before their lines of dialogue. Use a Narrators voice to describe the non-dialoge parts of the text to set the stage for the conversation. Your goal is to create a dialogue that is engaging and entertaining for the audience. The dialogue should be written in a way that is easy to read aloud and should be suitable for a teather audience . Be creative and have fun with this task! Good luck!
    
    Here is the input text you will be working with:

    <input_text>
    {text}
    </input_text>

    First, carefully read through the input text and identify the various characters and what is the Narrators parts. Think about tone and phrasing so that it would engage the audience.
    <scratchpad>
    Brainstorm creative ways to present this theather dialogue. Consider the personalities of the characters and how they might interact with each other. Identify parts that are not dialoge and use a Narrators voice for them. Think about the setting and how it can be used to enhance the dialogue. Jot down any ideas or key points that come to mind as you read through the input text. This will help you create a more engaging and entertaining dialogue. Be sure to include any important details or themes that you want to incorporate into the dialogue.
    </scratchpad>

    Now that you have brainstormed ideas and created a rough outline, it's time to write the actual play dialogue. Aim for a natural, conversational flow between the characters involved. Don't forget the Narrators voice for non dialoge parts. Incorporate the best ideas from your brainstorming session.
    <podcast_dialogue>
    Write your engaging, theater dialogue here, based on the key points and creative ideas you came up with during the brainstorming session. Use a tone that is suitable for the situations in the play; it is important that the audience thinks it to be believable -- it will be directly converted into audio.
    </podcast_dialogue>
    """
   
llm_output = generate_dialogue(text)


In [16]:
llm_output

Dialogue(scratchpad="This scene takes place in a dimly lit chamber on Mars, where Lorquas Ptomel, a green Martian leader, interrogates Dejah Thoris, a red Martian, who has been captured during a scientific expedition. The tension is palpable as Dejah attempts to appeal to Lorquas's sense of reason and unity between their peoples. The atmosphere is thick with moments of silence that suggest deep contemplation among the warriors present.", dialogue=[DialogueItem(text='Lorquas Ptomel raises his eyes to the prisoner, frustration brewing beneath his calm surface.', speaker='male-1'), DialogueItem(text='What is your name?', speaker='male-1'), DialogueItem(text='Dejah Thoris, daughter of Mors Kajak of Helium.', speaker='female-1'), DialogueItem(text='And the nature of your expedition?', speaker='male-1'), DialogueItem(text='It was a purely scientific research party sent out by my father’s father, the Jeddak of Helium, to rechart the air currents, and to take atmospheric density tests.', speak

In [13]:
def get_mp3(text: str, voice: str, api_key: str = None) -> bytes:
    client = OpenAI(
        api_key=api_key or os.getenv("OPENAI_API_KEY"),
    )

    with client.audio.speech.with_streaming_response.create(
        model="tts-1",
        voice=voice,
        input=text,
    ) as response:
        with io.BytesIO() as file:
            for chunk in response.iter_bytes():
                file.write(chunk)
            return file.getvalue()

In [14]:
audio = b""
transcript = ""

characters = 0

with cf.ThreadPoolExecutor() as executor:
    futures = []
    for line in llm_output.dialogue:
        transcript_line = f"{line.speaker}: {line.text}"
        future = executor.submit(get_mp3, line.text, line.voice, api_key)
        futures.append((future, transcript_line))
        characters += len(line.text)

    for future, transcript_line in futures:
        audio_chunk = future.result()
        audio += audio_chunk
        transcript += transcript_line + "\n\n"

In [15]:
temporary_directory = "./output/"
os.makedirs(temporary_directory, exist_ok=True)

temporary_file = NamedTemporaryFile(
    dir=temporary_directory,
    delete=False,
    suffix=".mp3",
)
temporary_file.write(audio)
temporary_file.close()